In [1]:
from google.colab import files
uploaded = files.upload()

Saving EmiratesReviews.csv to EmiratesReviews.csv


In [2]:
import pandas as pd

df = pd.read_csv("EmiratesReviews.csv")

## Format and Reorganize DataFrame Columns for Standardized Output

In [5]:
df_new = df.rename(columns={
    "Title": "header",
    "Date Published": "date",
    "Status": "trip_verified",  # Assuming 'Status' maps to 'trip_verified'
    "Aircraft": "aircraft",
    "Travel Type": "traveller_type",  # Renamed 'Travel Type' to 'traveller_type'
    "Travel Class": "seat_type",
    "Route": "route",
    "Date Flown": "date_flown",
    "Seating Comfort": "seat_comfort",
    "Staff Service": "cabin_staff_service",  # Renamed 'Staff Service' to 'cabin_staff_service'
    "Food Quality": "food_beverages",  # Renamed 'Food Quality' to 'food_beverages'
    "Entertainment": "entertainment",
    "WiFi": "wifi",  # Renamed WiFi to 'wifi'
    "Ground Service": "ground_service",
    "Value for Money": "value_for_money",
    "Recommended": "recommended",
    "Overall Rating": "rating",  # Renamed 'Overall Rating' to 'rating'
    "Review": "content",  # 'Review' is the content of the review
    "Country": "place"  # 'Country' is renamed to 'place'
})

# Add 'author' column with 'None' and replace it with 'traveller_type'
df_new["author"] = df_new["traveller_type"]

# Reordering the columns as per our# desired output
new_columns = [
    "header", "author", "date", "place", "content", "aircraft",
    "traveller_type", "seat_type", "route", "date_flown", "recommended",
    "trip_verified", "rating", "seat_comfort", "cabin_staff_service",
    "food_beverages", "ground_service", "value_for_money", "entertainment"
]

# Ensure the new order
df_new = df_new[new_columns]

#print(df_new.head(2))

## Ensure Numeric Consistency for Numeric Columns
### (Everything should be in float)

In [7]:
print(df_new.dtypes)

header                  object
author                  object
date                    object
place                   object
content                 object
aircraft                object
traveller_type          object
seat_type               object
route                   object
date_flown              object
recommended             object
trip_verified           object
rating                   int64
seat_comfort           float64
cabin_staff_service    float64
food_beverages         float64
ground_service         float64
value_for_money          int64
entertainment          float64
dtype: object


In [18]:
# Convert 'value_for_money' and 'rating' explicitly to float
df_new['value_for_money'] = pd.to_numeric(df_new['value_for_money'], errors='coerce')
df_new['rating'] = pd.to_numeric(df_new['rating'], errors='coerce')

# Select remaining numerical columns and explicitly convert them to float
numerical_columns = df_new.select_dtypes(include=['number']).columns

# Apply conversion to float for all numerical columns
df_new[numerical_columns] = df_new[numerical_columns].astype(float)


In [19]:
print(df_new.dtypes)

header                  object
author                  object
date                    object
place                   object
content                 object
aircraft                object
traveller_type          object
seat_type               object
route                   object
date_flown              object
recommended             object
trip_verified           object
rating                 float64
seat_comfort           float64
cabin_staff_service    float64
food_beverages         float64
ground_service         float64
value_for_money        float64
entertainment          float64
dtype: object


## Converting Date into required format

### (mm/dd/yy)

In [21]:
from datetime import datetime

# Function to clean and convert dates
def convert_date(date_str):
    try:
        # Remove ordinal suffixes (e.g., 'th', 'st', 'nd', 'rd')
        date_str = date_str.replace('th', '').replace('st', '').replace('nd', '').replace('rd', '')

        # Fix potential typos like "Augu" to "August"
        date_str = date_str.replace('Augu', 'August')

        # Parse and convert to the desired format
        return datetime.strptime(date_str.strip(), "%d %B %Y").strftime("%m/%d/%y")
    except Exception:
        return date_str  # Return original if parsing fails

# Apply the function to the 'date' column
df_new['date'] = df_new['date'].apply(convert_date)

# Copy the 'date' values to the 'date_flown' column
df_new['date_flown'] = df_new['date']

# Verify the changes
#print(df_new[['date', 'date_flown']].head())

## Standardize and Clean Aircraft Names in Dataset


In [25]:
import re

# Function to standardize aircraft names with additional cleanup
def standardize_aircraft(aircraft, mapping):
    # Remove everything after 'and' (case-insensitive)
    aircraft = re.sub(r"\s*and\s.*", "", aircraft, flags=re.IGNORECASE).strip()

    # Handle cases like "1 x Boeing 777 ER", removing the prefix "1 x"
    aircraft = re.sub(r"^\d+\s*x\s*", "", aircraft).strip()

    # Handle multiple aircraft types (e.g., "A380-800 / Boeing 777-300")
    if "/" in aircraft or "&" in aircraft or "," in aircraft:
        aircraft = aircraft.split("/")[0].split("&")[0].split(",")[0].strip()

    # Clean up spaces, hyphens, and make the name consistent
    aircraft = re.sub(r"[\s\-]+", " ", aircraft).strip()

    # Normalize aircraft names using the mapping
    for standard, variations in mapping.items():
        if aircraft.lower() in [v.lower() for v in variations]:
            return standard

    return aircraft.strip()

# Create the mapping dictionary
standard_aircraft = {
    "Boeing 777-300ER": ["boeing 777-300er", "b777-300er", "777-300er", "boeing777-300er", "boeing 777 300 er", "Boeing 777 300ER",
                         "Boeing 777 300er", "Boeing 777 300Er", "Boeing 777 300BR", "Boeing 773ER", "Boeing777 300er",
                         "777 300ER", "B777 300ER", "Boring 777 300ER", "B777", "Boing 777", "Boring 777", "Boeing 777 ER",
                         "Boeing B777 300ER", "Boeing 777 31H (ER)", "B777 300", "Boeing 777 300R", "Boeing 737 300",
                         "Boing 777 300", "Boeing 777‐300ER", "Boeing 777", "Boeing 777 300"],
    "A380-800": ["a380-800", "a380 800", "a388", "a380", "A 380", "A380 8", "A380 800", "Airbus A380", "AB380", "A380 80"],
    "Boeing 747": ["boeing 747", "boeing 747-400", "Boeing 747 400"],
    # Add more mappings as needed
}

# Drop rows with missing or invalid aircraft values
df_new = df_new.dropna(subset=["aircraft"])  # Drop rows with missing aircraft types
df_new = df_new[~df_new["aircraft"].isin(['?', ''])]  # Remove rows where aircraft column contains '?' or is empty

# Apply the cleaning function to standardize the aircraft names in-place
df_new["aircraft"] = df_new["aircraft"].apply(lambda x: standardize_aircraft(x, standard_aircraft))

# Check the cleaned data
print(df_new["aircraft"].value_counts())


aircraft
A380-800            631
Boeing 777-300ER    373
Boeing 777 200       10
A340                  3
Boeing 747            3
Boeing 737Max         1
A340 300              1
A330                  1
Boeing 77W            1
Ek227                 1
EK 203                1
Emirates              1
Boeing 777W           1
Boeing 777L           1
Boeing 777 200LR      1
BUS 780               1
A380 Boeing 777       1
BOEIN                 1
Boeing 772            1
777 A380              1
A330 200              1
Name: count, dtype: int64
